In [1]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import pandas as pd
from torch_geometric.data import Data
from torch_geometric.utils import negative_sampling

edges_df = pd.read_csv('GNN/protein_interactions.csv')

col_name = ['protein']
for i in range(1,769):
  col_name.append('feature'+str(i))
features_df = pd.read_csv('GNN/gene_embedding_GeneLLM_2.csv', header=None, names=col_name)

print(features_df)
labels_df = pd.read_csv('GNN/solubility.csv')

/tmp/ipykernel_1579328/4035422744.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/tmp/ipykernel_1579328/4035422744.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  edges_df = pd.read_csv('GNN/protein_interactions.csv')


      protein  feature1  feature2  feature3  feature4  feature5  feature6  \
0         FES  0.339602 -0.030744 -0.901381  0.100888  0.886443  0.383596   
1      HADHA  -0.131799 -0.025745 -0.677301 -0.053545  0.971046  0.180315   
2      SLC7A7  0.385693 -0.070692 -0.847796 -0.022054  0.959772  0.085487   
3        LCK   0.650428  0.014479 -0.866163  0.053508  0.951529  0.269402   
4       HSPA2  0.322262  0.017484 -0.849302  0.046401  0.920429  0.463832   
...       ...       ...       ...       ...       ...       ...       ...   
14445   BPY2C -0.840158 -0.042814 -0.853394 -0.049438  0.943925  0.104337   
14446    CLPS -0.270716 -0.036871 -0.915350 -0.013635  0.972046  0.016017   
14447    DNER  0.228932 -0.033579 -0.907262  0.010446  0.961684  0.524211   
14448    SOX7  0.140491  0.033339 -0.806014 -0.072016  0.938781  0.339959   
14449  CXCL14 -0.570266 -0.011502 -0.741149 -0.096209  0.967244  0.426519   

       feature7  feature8  feature9  ...  feature759  feature760  feature76

In [2]:
print(labels_df)
edges_df['combined_score'][7] = 594
edges_df_cleaned = edges_df.dropna()
print(edges_df_cleaned)
edges_df_cleaned = edges_df.dropna()
print(edges_df_cleaned)

     Gene name Solubility  Label  Word_Count Count_Category
0        ERAP2   Membrane      0         117     High Count
1     ADAMTSL5    Soluble      1          28      Low Count
2      TBC1D30   Membrane      0          55     High Count
3       KCNK18   Membrane      0         184     High Count
4         NDNF    Soluble      1         129     High Count
...        ...        ...    ...         ...            ...
1374   TRABD2B   Membrane      0          96     High Count
1375      RPS9    Soluble      1         205     High Count
1376  SLC22A16   Membrane      0          93     High Count
1377      FBN3    Soluble      1          90     High Count
1378      BDH2    Soluble      1         102     High Count

[1379 rows x 5 columns]


/tmp/ipykernel_1579328/3707133632.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  edges_df['combined_score'][7] = 594


         protein1  protein2 combined_score
0            ARF5   RALGPS2            173
1            ARF5     FHDC1            154
2            ARF5  ATP6V1E1            151
3            ARF5     CYTH2            471
4            ARF5      PSD3            201
...           ...       ...            ...
13715123     LDB1    SAMD14            260
13715124     LDB1     KDM6B            161
13715125     LDB1      WWP2            229
13715126     LDB1    VPS33B            152
13715127     LDB1     NDST2            440

[13067419 rows x 3 columns]
         protein1  protein2 combined_score
0            ARF5   RALGPS2            173
1            ARF5     FHDC1            154
2            ARF5  ATP6V1E1            151
3            ARF5     CYTH2            471
4            ARF5      PSD3            201
...           ...       ...            ...
13715123     LDB1    SAMD14            260
13715124     LDB1     KDM6B            161
13715125     LDB1      WWP2            229
13715126     LDB1    VPS3

In [3]:
node_id_f = [node_id for node_id in features_df['protein']]
node_id_e1 = [node_id for node_id in edges_df_cleaned['protein1']]
node_id_e2 = [node_id for node_id in edges_df_cleaned['protein2']]
node_id_e = list(set(node_id_e1 + node_id_e2))
print(len(node_id_f))
print(len(node_id_e))

14450
18838


In [4]:
labels_df.rename(columns={'Gene name': 'protein'}, inplace=True)
labels_df

,protein,Solubility,Label,Word_Count,Count_Category
0,ERAP2,Membrane,0,117,High Count
1,ADAMTSL5,Soluble,1,28,Low Count
2,TBC1D30,Membrane,0,55,High Count
3,KCNK18,Membrane,0,184,High Count
4,NDNF,Soluble,1,129,High Count
...,...,...,...,...,...
1374,TRABD2B,Membrane,0,96,High Count
1375,RPS9,Soluble,1,205,High Count
1376,SLC22A16,Membrane,0,93,High Count
1377,FBN3,Soluble,1,90,High Count


In [5]:
nodes_in_edges = set(edges_df['protein1']).union(set(edges_df['protein2']))
nodes_in_features = set(features_df['protein'])
missing_nodes = list(nodes_in_edges - nodes_in_features)
average_features = features_df.iloc[:, 1:].mean()

# 为缺失的节点创建平均特征
missing_features =pd.DataFrame([average_features.values] * len(missing_nodes), columns=average_features.index)
missing_features['protein'] = missing_nodes
missing_features = missing_features[features_df.columns]

# 合并原始特征和缺失节点的特征
new_features_df = pd.concat([features_df, missing_features], ignore_index=True)
new_features_df

,protein,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature759,feature760,feature761,feature762,feature763,feature764,feature765,feature766,feature767,feature768
0,FES,0.339602,-0.030744,-0.901381,0.100888,0.886443,0.383596,-0.192082,-0.032063,-0.154869,...,-0.549204,-0.856123,0.714672,-0.046649,-0.894424,-0.001815,0.739485,0.015581,-0.023863,-0.022002
1,HADHA,-0.131799,-0.025745,-0.677301,-0.053545,0.971046,0.180315,-0.028189,-0.077389,-0.095152,...,0.927885,-0.817812,0.809631,-0.005827,-0.848839,0.024516,0.526404,-0.039926,-0.102787,-0.026980
2,SLC7A7,0.385693,-0.070692,-0.847796,-0.022054,0.959772,0.085487,0.076455,-0.003006,-0.032268,...,0.941094,-0.912443,0.789828,0.046979,-0.715636,0.085842,0.150494,0.025392,-0.066035,-0.028283
3,LCK,0.650428,0.014479,-0.866163,0.053508,0.951529,0.269402,-0.214788,0.045179,-0.506429,...,-0.576739,-0.969558,0.916549,-0.080332,-0.927649,-0.047398,0.741663,-0.000096,-0.096318,-0.056501
4,HSPA2,0.322262,0.017484,-0.849302,0.046401,0.920429,0.463832,-0.050414,-0.033398,0.387791,...,0.387301,-0.860696,0.678607,-0.060695,-0.945793,0.040472,0.831079,-0.001711,-0.079842,-0.011189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19196,C5orf63,0.166824,-0.022353,-0.818924,-0.015590,0.938668,0.350558,-0.064741,-0.024466,0.048526,...,0.638018,-0.851449,0.644471,-0.036707,-0.842413,0.005465,0.456407,-0.028600,-0.076502,-0.027952
19197,PRRX1,0.166824,-0.022353,-0.818924,-0.015590,0.938668,0.350558,-0.064741,-0.024466,0.048526,...,0.638018,-0.851449,0.644471,-0.036707,-0.842413,0.005465,0.456407,-0.028600,-0.076502,-0.027952
19198,KRTAP13-2,0.166824,-0.022353,-0.818924,-0.015590,0.938668,0.350558,-0.064741,-0.024466,0.048526,...,0.638018,-0.851449,0.644471,-0.036707,-0.842413,0.005465,0.456407,-0.028600,-0.076502,-0.027952
19199,DRC1,0.166824,-0.022353,-0.818924,-0.015590,0.938668,0.350558,-0.064741,-0.024466,0.048526,...,0.638018,-0.851449,0.644471,-0.036707,-0.842413,0.005465,0.456407,-0.028600,-0.076502,-0.027952


In [6]:
# 从边数据集获取节点
nodes_in_edges = set(edges_df_cleaned['protein1']).union(set(edges_df_cleaned['protein2']))

# 从特征数据集获取节点
nodes_in_features = set(features_df['protein'])

# 找出共同的节点
common_nodes = nodes_in_edges.intersection(nodes_in_features)

# 过滤边数据集，保留只包含共同节点的边
filtered_edges_df = edges_df_cleaned[edges_df_cleaned['protein1'].isin(common_nodes) & edges_df_cleaned['protein2'].isin(common_nodes)].reset_index(drop=True)

# 过滤特征数据集，保留共同节点的特征
filtered_features_df = features_df[features_df['protein'].isin(common_nodes)].reset_index(drop=True)

In [7]:
filtered_edges_df

,protein1,protein2,combined_score
0,ARF5,RALGPS2,173
1,ARF5,FHDC1,154
2,ARF5,ATP6V1E1,151
3,ARF5,CYTH2,471
4,ARF5,PSD3,201
...,...,...,...
9503498,LDB1,PGAP6,197
9503499,LDB1,KDM6B,161
9503500,LDB1,WWP2,229
9503501,LDB1,VPS33B,152


In [8]:
filtered_features_df

,protein,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature759,feature760,feature761,feature762,feature763,feature764,feature765,feature766,feature767,feature768
0,FES,0.339602,-0.030744,-0.901381,0.100888,0.886443,0.383596,-0.192082,-0.032063,-0.154869,...,-0.549204,-0.856123,0.714672,-0.046649,-0.894424,-0.001815,0.739485,0.015581,-0.023863,-0.022002
1,SLC7A7,0.385693,-0.070692,-0.847796,-0.022054,0.959772,0.085487,0.076455,-0.003006,-0.032268,...,0.941094,-0.912443,0.789828,0.046979,-0.715636,0.085842,0.150494,0.025392,-0.066035,-0.028283
2,HSPA2,0.322262,0.017484,-0.849302,0.046401,0.920429,0.463832,-0.050414,-0.033398,0.387791,...,0.387301,-0.860696,0.678607,-0.060695,-0.945793,0.040472,0.831079,-0.001711,-0.079842,-0.011189
3,ZSCAN9,-0.132130,-0.048746,-0.844795,0.013757,0.963150,0.614866,-0.126746,-0.017795,-0.464897,...,0.987995,-0.719372,0.739748,0.122923,-0.982124,-0.104269,0.696067,-0.015833,-0.163702,0.037182
4,SNX10,0.418061,-0.084858,-0.917636,-0.053882,0.942593,0.590945,-0.142904,-0.005636,-0.006018,...,0.768108,-0.841070,0.306929,-0.030281,-0.918397,0.043871,0.892007,0.010785,-0.074765,-0.066948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14083,IQCJ-SCHIP1,0.092851,-0.004035,-0.841137,-0.094960,0.741134,-0.082558,-0.051283,-0.074935,0.052519,...,0.836989,-0.510768,0.568964,-0.120246,-0.577445,0.002407,-0.329807,-0.055330,-0.065028,0.007841
14084,CLPS,-0.270716,-0.036871,-0.915350,-0.013635,0.972046,0.016017,-0.018190,-0.008408,0.548612,...,0.946926,-0.739839,0.755471,0.006047,-0.488539,0.009771,-0.261908,-0.048952,-0.062768,0.019626
14085,DNER,0.228932,-0.033579,-0.907262,0.010446,0.961684,0.524211,-0.214318,0.039835,0.433141,...,0.806205,-0.857853,0.223238,-0.078767,-0.973104,0.026339,0.918282,-0.041604,-0.039557,-0.014415
14086,SOX7,0.140491,0.033339,-0.806014,-0.072016,0.938781,0.339959,-0.045201,-0.004075,-0.169557,...,0.756285,-0.889968,0.837899,-0.030372,-0.969834,-0.054169,0.099373,0.018777,-0.166461,-0.062177


In [9]:
# 将第二列中的两种字符映射为0和1
unique_values = labels_df['Count_Category'].unique()
print(unique_values)
mapping = {unique_values[0]: 0, unique_values[1]: 1}

labels_df['Count_Category'] = labels_df['Count_Category'].map(mapping)
# 显示更新后的DataFrame
print(labels_df)

# 如果需要，可以将更新后的DataFrame保存回CSV文件
#labels_df.to_csv('/content/drive/MyDrive/GNN/updated_file.csv', index=False)

['High Count' 'Low Count']
       protein Solubility  Label  Word_Count  Count_Category
0        ERAP2   Membrane      0         117               0
1     ADAMTSL5    Soluble      1          28               1
2      TBC1D30   Membrane      0          55               0
3       KCNK18   Membrane      0         184               0
4         NDNF    Soluble      1         129               0
...        ...        ...    ...         ...             ...
1374   TRABD2B   Membrane      0          96               0
1375      RPS9    Soluble      1         205               0
1376  SLC22A16   Membrane      0          93               0
1377      FBN3    Soluble      1          90               0
1378      BDH2    Soluble      1         102               0

[1379 rows x 5 columns]


In [10]:
node_id_to_index = {node_id: i for i, node_id in enumerate(filtered_features_df['protein'])}
# 确保edge_index是按照这个新的索引顺序排列的
source_indices = [node_id_to_index[node_id] for node_id in filtered_edges_df['protein1']]
target_indices = [node_id_to_index[node_id] for node_id in filtered_edges_df['protein2']]
edge_index = torch.tensor([source_indices, target_indices], dtype=torch.long)
filtered_edges_df['combined_score'] = pd.to_numeric(filtered_edges_df['combined_score'], errors='coerce', downcast='float')
edge_weight = torch.tensor(filtered_edges_df['combined_score'].values, dtype=torch.float)

# 重排特征矩阵以匹配edge_index的顺序
features = filtered_features_df.iloc[:, 1:].values
#features = features[[node_id_to_index[node_id] for node_id in filtered_features_df['protein']], :]

# 转换为PyTorch张量
features_tensor = torch.tensor(features, dtype=torch.float)

In [11]:
filtered_features_df

,protein,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature759,feature760,feature761,feature762,feature763,feature764,feature765,feature766,feature767,feature768
0,FES,0.339602,-0.030744,-0.901381,0.100888,0.886443,0.383596,-0.192082,-0.032063,-0.154869,...,-0.549204,-0.856123,0.714672,-0.046649,-0.894424,-0.001815,0.739485,0.015581,-0.023863,-0.022002
1,SLC7A7,0.385693,-0.070692,-0.847796,-0.022054,0.959772,0.085487,0.076455,-0.003006,-0.032268,...,0.941094,-0.912443,0.789828,0.046979,-0.715636,0.085842,0.150494,0.025392,-0.066035,-0.028283
2,HSPA2,0.322262,0.017484,-0.849302,0.046401,0.920429,0.463832,-0.050414,-0.033398,0.387791,...,0.387301,-0.860696,0.678607,-0.060695,-0.945793,0.040472,0.831079,-0.001711,-0.079842,-0.011189
3,ZSCAN9,-0.132130,-0.048746,-0.844795,0.013757,0.963150,0.614866,-0.126746,-0.017795,-0.464897,...,0.987995,-0.719372,0.739748,0.122923,-0.982124,-0.104269,0.696067,-0.015833,-0.163702,0.037182
4,SNX10,0.418061,-0.084858,-0.917636,-0.053882,0.942593,0.590945,-0.142904,-0.005636,-0.006018,...,0.768108,-0.841070,0.306929,-0.030281,-0.918397,0.043871,0.892007,0.010785,-0.074765,-0.066948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14083,IQCJ-SCHIP1,0.092851,-0.004035,-0.841137,-0.094960,0.741134,-0.082558,-0.051283,-0.074935,0.052519,...,0.836989,-0.510768,0.568964,-0.120246,-0.577445,0.002407,-0.329807,-0.055330,-0.065028,0.007841
14084,CLPS,-0.270716,-0.036871,-0.915350,-0.013635,0.972046,0.016017,-0.018190,-0.008408,0.548612,...,0.946926,-0.739839,0.755471,0.006047,-0.488539,0.009771,-0.261908,-0.048952,-0.062768,0.019626
14085,DNER,0.228932,-0.033579,-0.907262,0.010446,0.961684,0.524211,-0.214318,0.039835,0.433141,...,0.806205,-0.857853,0.223238,-0.078767,-0.973104,0.026339,0.918282,-0.041604,-0.039557,-0.014415
14086,SOX7,0.140491,0.033339,-0.806014,-0.072016,0.938781,0.339959,-0.045201,-0.004075,-0.169557,...,0.756285,-0.889968,0.837899,-0.030372,-0.969834,-0.054169,0.099373,0.018777,-0.166461,-0.062177


In [12]:
filtered_edges_df

,protein1,protein2,combined_score
0,ARF5,RALGPS2,173.0
1,ARF5,FHDC1,154.0
2,ARF5,ATP6V1E1,151.0
3,ARF5,CYTH2,471.0
4,ARF5,PSD3,201.0
...,...,...,...
9503498,LDB1,PGAP6,197.0
9503499,LDB1,KDM6B,161.0
9503500,LDB1,WWP2,229.0
9503501,LDB1,VPS33B,152.0


In [13]:
edge_index

tensor([[11049, 11049, 11049,  ...,  9882,  9882,  9882],
        [13911, 12214, 11090,  ...,   910,  6714, 12904]])

In [14]:
edge_weight

tensor([173., 154., 151.,  ..., 229., 152., 440.])

In [15]:
features_tensor 

tensor([[ 0.3396, -0.0307, -0.9014,  ...,  0.0156, -0.0239, -0.0220],
        [ 0.3857, -0.0707, -0.8478,  ...,  0.0254, -0.0660, -0.0283],
        [ 0.3223,  0.0175, -0.8493,  ..., -0.0017, -0.0798, -0.0112],
        ...,
        [ 0.2289, -0.0336, -0.9073,  ..., -0.0416, -0.0396, -0.0144],
        [ 0.1405,  0.0333, -0.8060,  ...,  0.0188, -0.1665, -0.0622],
        [-0.5703, -0.0115, -0.7411,  ..., -0.0046,  0.0743, -0.0935]])

In [16]:
print(len(labels_df))
nodes_in_labels = set(labels_df['protein'])
nodes_in_filter_features = set(filtered_features_df['protein'])
common_nodes_labels = nodes_in_labels.intersection(nodes_in_filter_features)

# 过滤
filtered_labels_df = labels_df[labels_df['protein'].isin(common_nodes_labels)].reset_index(drop=True)
print(len(filtered_labels_df))
print(len(features_tensor))

1379
1355
14088


In [17]:
labels_df

,protein,Solubility,Label,Word_Count,Count_Category
0,ERAP2,Membrane,0,117,0
1,ADAMTSL5,Soluble,1,28,1
2,TBC1D30,Membrane,0,55,0
3,KCNK18,Membrane,0,184,0
4,NDNF,Soluble,1,129,0
...,...,...,...,...,...
1374,TRABD2B,Membrane,0,96,0
1375,RPS9,Soluble,1,205,0
1376,SLC22A16,Membrane,0,93,0
1377,FBN3,Soluble,1,90,0


In [18]:
filtered_labels_df

,protein,Solubility,Label,Word_Count,Count_Category
0,ERAP2,Membrane,0,117,0
1,ADAMTSL5,Soluble,1,28,1
2,TBC1D30,Membrane,0,55,0
3,KCNK18,Membrane,0,184,0
4,NDNF,Soluble,1,129,0
...,...,...,...,...,...
1350,TRABD2B,Membrane,0,96,0
1351,RPS9,Soluble,1,205,0
1352,SLC22A16,Membrane,0,93,0
1353,FBN3,Soluble,1,90,0


In [19]:
#num_nodes = 14088
#labels = torch.full((num_nodes,), -1, dtype=torch.long)
label_indices = [node_id_to_index[node_id] for node_id in filtered_labels_df['protein']]
print(label_indices)

[61, 75, 87, 88, 94, 109, 122, 135, 139, 140, 148, 157, 160, 192, 218, 225, 229, 231, 243, 246, 264, 269, 275, 282, 292, 305, 307, 319, 325, 354, 358, 360, 366, 367, 380, 383, 398, 422, 430, 449, 458, 459, 465, 473, 485, 492, 500, 532, 543, 559, 560, 561, 567, 568, 569, 570, 573, 588, 614, 625, 626, 636, 654, 707, 724, 732, 734, 743, 747, 751, 800, 817, 858, 859, 863, 878, 880, 901, 907, 920, 945, 959, 965, 981, 1002, 1009, 1015, 1024, 1025, 1026, 1028, 1035, 1036, 1039, 1040, 1066, 1069, 1076, 1077, 1080, 1084, 1091, 1096, 1097, 1103, 1117, 1130, 1133, 1143, 1154, 1155, 1156, 1174, 1193, 1194, 1200, 1213, 1218, 1219, 1229, 1230, 1232, 1264, 1268, 1279, 1285, 1299, 1314, 1322, 1328, 1329, 1334, 1343, 1348, 1377, 1378, 1382, 1386, 1395, 1396, 1419, 1420, 1425, 1434, 1439, 1442, 1443, 1470, 1476, 1489, 1497, 1508, 1524, 1542, 1555, 1558, 1563, 1565, 1567, 1585, 1586, 1593, 1622, 1629, 1658, 1668, 1681, 1684, 1685, 1706, 1723, 1742, 1754, 1770, 1779, 1780, 1790, 1794, 1795, 1798, 1800, 18

In [20]:
num_nodes = 14088
labels = torch.full((num_nodes,), -1, dtype=torch.long)
for i, index in enumerate(filtered_labels_df['Label']):
    labels[label_indices[i]] = index

labels_tensor = torch.tensor(labels, dtype=torch.long)

/tmp/ipykernel_1579328/1869846290.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_tensor = torch.tensor(labels, dtype=torch.long)


In [21]:
highinfo_indices = [
    node_id_to_index[node_id]
    for node_id, weight in zip(filtered_labels_df['protein'], filtered_labels_df['Count_Category'])
    if weight == 0
]
print(highinfo_indices)
lowinfo_indices = [
    node_id_to_index[node_id]
    for node_id, weight in zip(filtered_labels_df['protein'], filtered_labels_df['Count_Category'])
    if weight == 1
]
print(lowinfo_indices)


[61, 87, 88, 94, 109, 122, 135, 139, 148, 157, 218, 225, 229, 243, 264, 275, 282, 307, 319, 325, 354, 358, 360, 366, 367, 380, 383, 398, 422, 449, 458, 465, 473, 485, 492, 500, 532, 543, 560, 561, 567, 568, 569, 570, 573, 588, 614, 626, 636, 654, 707, 724, 734, 747, 800, 817, 858, 859, 863, 878, 880, 901, 907, 920, 945, 959, 965, 981, 1002, 1015, 1024, 1025, 1026, 1028, 1035, 1036, 1039, 1040, 1066, 1069, 1076, 1077, 1080, 1084, 1091, 1096, 1097, 1103, 1117, 1130, 1133, 1143, 1154, 1155, 1156, 1174, 1193, 1194, 1200, 1213, 1218, 1229, 1230, 1232, 1264, 1268, 1279, 1285, 1299, 1314, 1328, 1329, 1343, 1348, 1378, 1395, 1396, 1419, 1420, 1425, 1434, 1439, 1442, 1443, 1470, 1497, 1508, 1524, 1542, 1558, 1563, 1565, 1567, 1585, 1586, 1622, 1629, 1668, 1681, 1684, 1685, 1706, 1723, 1742, 1754, 1779, 1780, 1790, 1795, 1798, 1800, 1825, 1836, 1838, 1841, 1857, 1860, 1866, 1901, 1911, 1913, 1918, 1926, 1935, 1955, 1970, 1985, 1987, 2010, 2033, 2034, 2038, 2046, 2051, 2053, 2073, 2084, 2087, 209

In [22]:
data = Data(x=features_tensor, edge_index=edge_index, y=labels_tensor, edge_attr=edge_weight)

print("x:", data.x.shape, data.x.dtype)
print("edge_index:", data.edge_index.shape, data.edge_index.dtype)
print("labels:", data.y.shape, data.y.dtype)
print("edge_weight:", data.edge_attr.shape, data.edge_attr.dtype)

x: torch.Size([14088, 768]) torch.float32
edge_index: torch.Size([2, 9503503]) torch.int64
labels: torch.Size([14088]) torch.int64
edge_weight: torch.Size([9503503]) torch.float32


In [23]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
class GCN(torch.nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 128)
        self.conv2 = GCNConv(128, hidden_dim)
        self.fc1 = torch.nn.Linear(hidden_dim, 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, num_classes)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x, edge_index, edge_weight):
        # 保存初始特征
        initial_features = x

        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)

        # 在这里将初始特征和 GCN 的输出拼接在一起
        #x = torch.cat([x, initial_features], dim=1)

        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sigmoid(x)  # 应用 Sigmoid 激活函数进行逻辑回归
        return x

In [24]:
device = torch.device('cuda')
data = data.to(device)

model = GCN(num_features=features.shape[1], hidden_dim=64, num_classes=1).to(device)

criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

In [25]:
label_indices = torch.tensor(label_indices, dtype=torch.long)
print(label_indices)
# 随机打乱有标签的节点索引
labeled_indices = label_indices[torch.randperm(label_indices.size(0))]
#print(labeled_indices)
labeled_indices = label_indices

# 定义训练和测试集的大小
num_labeled = labeled_indices.size(0)
num_train = int(num_labeled * 0.8)
num_test = num_labeled - num_train
print(num_test)

# 创建训练和测试掩码
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

train_mask[labeled_indices[:num_train]] = True
test_mask[labeled_indices[num_train:num_train+num_test]] = True
print(test_mask)

tensor([   61,    75,    87,  ..., 14038, 14047, 14066])
271
tensor([False, False, False,  ..., False, False, False])


In [26]:
train_mask

tensor([False, False, False,  ..., False, False, False])

In [27]:
# 训练模型
model.train()
for epoch in range(3000):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_attr).squeeze()
    loss = criterion(out[train_mask], data.y[train_mask].float())
    loss.backward()
    optimizer.step()
    
    if epoch % 50 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 0, Loss: 0.6915607452392578
Epoch 50, Loss: 0.6328170299530029
Epoch 100, Loss: 0.6143164038658142
Epoch 150, Loss: 0.5972852110862732
Epoch 200, Loss: 0.5903028845787048
Epoch 250, Loss: 0.7974728345870972
Epoch 300, Loss: 0.6187817454338074
Epoch 350, Loss: 0.5903077125549316
Epoch 400, Loss: 0.5758543014526367
Epoch 450, Loss: 0.5718207955360413
Epoch 500, Loss: 0.5469692945480347
Epoch 550, Loss: 0.5499712228775024
Epoch 600, Loss: 0.5302851796150208
Epoch 650, Loss: 0.5694785118103027
Epoch 700, Loss: 0.5517493486404419
Epoch 750, Loss: 0.49910756945610046
Epoch 800, Loss: 0.6017716526985168
Epoch 850, Loss: 0.49118369817733765
Epoch 900, Loss: 0.4835301339626312
Epoch 950, Loss: 0.4728066027164459
Epoch 1000, Loss: 0.4683257043361664
Epoch 1050, Loss: 0.46387267112731934
Epoch 1100, Loss: 0.5070878267288208
Epoch 1150, Loss: 0.5201672911643982
Epoch 1200, Loss: 0.44848182797431946
Epoch 1250, Loss: 0.46856528520584106
Epoch 1300, Loss: 0.548713207244873
Epoch 1350, Loss: 0.

In [28]:
high_indices = torch.tensor(highinfo_indices, dtype=torch.long)
high_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
high_mask[high_indices] = True
test_high = high_mask & test_mask
low_indices = torch.tensor(lowinfo_indices, dtype=torch.long)
low_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
low_mask[low_indices] = True
test_low = low_mask & test_mask

model.eval()
with torch.no_grad():
    probabilities = model(data.x, data.edge_index, data.edge_attr).squeeze()
    p = (probabilities[test_mask] > 0.5).cpu().numpy()  # 移动到 CPU 后再转换
    true = data.y[test_mask].cpu().numpy()
    predicted_classes_h = (probabilities[test_high] > 0.5).cpu().numpy()  # 移动到 CPU 后再转换
    true_classes_h = data.y[test_high].cpu().numpy()
    predicted_classes_l = (probabilities[test_low] > 0.5).cpu().numpy()  # 移动到 CPU 后再转换
    true_classes_l = data.y[test_low].cpu().numpy()
    import numpy as np
    accuracy = np.mean(p == true)
    accuracy_h = np.mean(predicted_classes_h == true_classes_h)
    accuracy_l = np.mean(predicted_classes_l == true_classes_l)
    print(f'Accuracy: {accuracy}')
    print(f'Accuracy_h: {accuracy_h}')
    print(f'Accuracy_l: {accuracy_l}')

Accuracy: 0.7785977859778598
Accuracy_h: 0.7946428571428571
Accuracy_l: 0.7021276595744681


In [29]:
import numpy as np
np.mean([0.8339,0.775,0.8229,0.8155,0.8229,0.79,0.7712,0.8007,0.7823,0.8229])

0.80373

In [30]:
np.mean([0.8491,0.7991,0.8526,0.8348,0.8482,0.817,0.7858,0.8304,0.8036,0.8482])

0.8268800000000001

In [31]:
np.mean([0.7436,0.6596,0.6809,0.7234,0.7021,0.6596,0.7021,0.66,0.6809,0.7021])

0.69143

In [37]:
import numpy as np
from sklearn.model_selection import KFold

def train(model, optimizer, data, train_mask, criterion):
    model.train()  # 设置模型为训练模式
    optimizer.zero_grad()  # 清除旧的梯度
    out = model(data.x, data.edge_index, data.edge_attr).squeeze()  # 前向传播
    loss = criterion(out[train_mask], data.y[train_mask].float())  # 计算损失
    loss.backward()  # 反向传播
    optimizer.step()  # 参数更新
    return loss.item()  # 返回损失值

def evaluate(model, data, test_mask, lowinfo_indices, highinfo_indices):
    high_indices = torch.tensor(highinfo_indices, dtype=torch.long)
    high_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    high_mask[high_indices] = True
    test_high = high_mask & test_mask
    low_indices = torch.tensor(lowinfo_indices, dtype=torch.long)
    low_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    low_mask[low_indices] = True
    test_low = low_mask & test_mask
    model.eval()  # 设置模型为评估模式
    with torch.no_grad():  # 不计算梯度
        probabilities = model(data.x, data.edge_index, data.edge_attr).squeeze()
        p = (probabilities[test_mask] > 0.5).cpu().numpy()  # 总体预测
        true = data.y[test_mask].cpu().numpy()
        predicted_classes_h = (probabilities[test_high] > 0.5).cpu().numpy()  # 高信息预测
        true_classes_h = data.y[test_high].cpu().numpy()
        predicted_classes_l = (probabilities[test_low] > 0.5).cpu().numpy()  # 低信息预测
        true_classes_l = data.y[test_low].cpu().numpy()
    
    # 计算准确率
    accuracy = np.mean(p == true)
    accuracy_h = np.mean(predicted_classes_h == true_classes_h)
    accuracy_l = np.mean(predicted_classes_l == true_classes_l)
    
    return accuracy, accuracy_h, accuracy_l


# 假设 labeled_indices 是所有有标签数据点的索引
num_labeled = labeled_indices.size(0)

# 使用 KFold 从 sklearn 进行数据分割
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)  # 可以设置 random_state 以确保可重复性

accuracy = []  # 存储每一折的准确率
accuracy_h = []
accuracy_l = []

for fold, (train_idx, test_idx) in enumerate(kf.split(labeled_indices)):
    # 根据当前折的分割，更新训练和测试掩码
    train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

    train_mask[labeled_indices[train_idx]] = True
    test_mask[labeled_indices[test_idx]] = True

    # 在这里重置模型并进行训练和评估
    model = GCN(num_features=features.shape[1], hidden_dim=64, num_classes=1).to(device)
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

    for epoch in range(3000):
        train_loss = train(model, optimizer, data, train_mask, criterion)
    
    # 评估模型
    acc, acc_h, acc_l = evaluate(model, data, test_mask, lowinfo_indices, highinfo_indices)
    accuracy.append(acc)
    accuracy_l.append(acc_l)
    accuracy_h.append(acc_h)
    print(f'Fold {fold+1}, Accuracy: {acc}')
    print(f'Fold {fold+1}, Accuracy of low information: {acc_l}')
    print(f'Fold {fold+1}, Accuracy of high information: {acc_h}')

average_accuracy = sum(accuracy) / k_folds
print(f'Average Accuracy: {average_accuracy}')
print(f'Average Accuracy of lowinfo: {sum(accuracy_l) / k_folds}')
print(f'Average Accuracy of highinfo: {sum(accuracy_h) / k_folds}')

Fold 1, Accuracy: 0.8007380073800738
Fold 1, Accuracy of low information: 0.7631578947368421
Fold 1, Accuracy of high information: 0.8068669527896996
Fold 2, Accuracy: 0.7970479704797048
Fold 2, Accuracy of low information: 0.7666666666666667
Fold 2, Accuracy of high information: 0.8008298755186722
Fold 3, Accuracy: 0.8044280442804428
Fold 3, Accuracy of low information: 0.868421052631579
Fold 3, Accuracy of high information: 0.7939914163090128
Fold 4, Accuracy: 0.8081180811808119
Fold 4, Accuracy of low information: 0.7878787878787878
Fold 4, Accuracy of high information: 0.8109243697478992
Fold 5, Accuracy: 0.8339483394833949
Fold 5, Accuracy of low information: 0.90625
Fold 5, Accuracy of high information: 0.8242677824267782
Average Accuracy: 0.8088560885608855
Average Accuracy of lowinfo: 0.8184748803827752
Average Accuracy of highinfo: 0.8073760793584125
